In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
df = pd.read_csv("kw_email_use_selenium.csv")

url_list = df['agent_url'].unique()
def get_driver():
    options = webdriver.FirefoxOptions()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Firefox(options=options)
    return driver


def collect_agent_contact_details(driver):
    try:
        agent_info = driver.find_element_by_class_name("AgentInformation")
    except:
        agent_info = ""
    try:
        info_container = agent_info.find_elements_by_class_name("AgentInformation__factBody")
    except:
        info_container = ""
    try:
        agent_email = [email.text for email in info_container if email.get_attribute("aria-label") == 'Agent E-mail'][0]
    except:
        agent_email = ""
    try:
        mobile_number = driver.find_element_by_class_name("AgentInformation__phoneMobileNumber").text
    except:
        mobile_number = ""
    try:
        office_number = driver.find_element_by_class_name("AgentInformation__phoneOfficeNumber").text
    except:
        office_number = ""
    try:
        agent_website = [i.text for i in info_container if i.get_attribute("aria-label") == 'Agent Website'][0]
    except:
        agent_website = ""
    contact_dict = {"agent_email":agent_email,"mobile_number":mobile_number,"office_number":office_number,'agent_website':agent_website }
    return contact_dict


def get_agent_details(driver):
    try:
        agent_licenses = driver.find_element_by_class_name("AgentContent__licenses").text
    except:
        agent_licenses = ""
    try:
        agent_bio = driver.find_element_by_class_name("AgentContent__bio").text
    except:
        agent_bio = ""
    try:
        serviceAreas = driver.find_element_by_class_name("AgentContent__serviceAreas").text
    except:
        serviceAreas = ""
    try:
        agent_team_name = driver.find_element_by_class_name("AgentContent__teamName").text
    except:
        agent_team_name = ""
    try:
        agent_team_info_ = driver.find_element_by_class_name("AgentContent__teamText").text.split("\n")
        agent_team_info = ", ".join(agent_team_info_)
    except:
        agent_team_info = ""
    try:
        logo_url = driver.find_element_by_class_name('AgentContent__teamAvatar').find_element_by_class_name("KWImage__image").get_attribute("src")
    except:
        logo_url = ""
    agent_details = {"agent_licenses":agent_licenses, "agent_bio":agent_bio, "serviceAreas":serviceAreas,"agent_team_name":agent_team_name,
                    "agent_team_info":agent_team_info,'logo_url':logo_url}
    return agent_details


def social_media_details(driver):
    try:
        social_media_container = driver.find_element_by_class_name("AgentInformation__socialMedia").find_elements_by_class_name("link")
    except:
        social_media_container = []
    social_media_links = [social_media.get_attribute("href") for social_media in social_media_container]
    try:
        facebook = [facebook for facebook in social_media_links if 'facebook' in facebook][0]
    except:
        facebook = ""
    try:
        instagram = [instagram for instagram in social_media_links if 'instagram' in instagram][0]
    except:
        instagram = ""
    try:
        twitter = [twitter for twitter in social_media_links if 'twitter' in twitter][0]
    except:
        twitter = ""
    try:
        linkedin = [linkedin for linkedin in social_media_links if 'linkedin' in linkedin][0]
    except:
        linkedin = ""
    social_media_dict = {"facebook":facebook,"instagram":instagram,"twitter":twitter,"linkedin":linkedin}
    social_media_links_str = "|".join(social_media_links)
    return social_media_dict,social_media_links_str


def collect_other_info(driver):
    container = driver.find_elements_by_class_name("AgentContent__section")
    try:
        Market_Cente = [i for i in container if 'Market Center' in i.text][0].find_element_by_class_name("AgentContent__sectionText").text
    except:
        Market_Cente = ""
    try:
        Languages = [i for i in container if 'Languages' in i.text][0].find_element_by_class_name("AgentContent__sectionText").text
    except:
        Languages = ""
    try:
        Specialties_designation = [i for i in container if 'Specialties and Designations' in i.text][0].find_element_by_class_name("AgentContent__sectionText").text
    except:
        Specialties_designation = ""
    other_info = {"Market_Cente":Market_Cente, "Languages":Languages, "Specialties_Designations":Specialties_designation}
    return other_info

def wait_to_page_load(driver):
    try:
        w = WebDriverWait(driver, 3)
        w.until(EC.presence_of_element_located((By.CLASS_NAME,"AgentContent__name")))
        print("Page load happened")
    except Exception as e:
        print("Timeout happened no page load")
    return driver

url_count = 43000
driver_count = 0
driver = get_driver()
for url in url_list[url_count:44000]:
    print(url)
    if driver_count >=100:
        print("need to close driver")
        driver.quit()
        driver_count = 0
        driver = get_driver()
        print("new driver initialize****************")
    driver.get(url)
    driver = wait_to_page_load(driver)
    try:
        agent_name = driver.find_element_by_class_name("AgentContent__name").text
    except:
        agent_name = ""
    try:
        time.sleep(0.5)
        profile_url = driver.find_element_by_class_name("AvatarImage__bg").get_attribute("style").replace('background-image: url("','').replace('");','')
    except:
        profile_url = ""
    try:
        tag = driver.find_element_by_class_name("pill").text
    except:
        tag = ""
    try:
        role = driver.find_element_by_class_name("AgentContent__team").text
    except:
        role = ""
    try:
        location = driver.find_element_by_class_name("AgentContent__location").text
    except:
        location = ""
    contact_dict = collect_agent_contact_details(driver)
    agent_details = get_agent_details(driver)
    social_media_dict,social_media_links_str = social_media_details(driver)
    other_info = collect_other_info(driver)
    data_dict = {'agent_url':url,"agent_name":agent_name, 'agent_role':role,'location':location,"agent_dp":profile_url, "tag":tag,"agent_email":contact_dict['agent_email'], "mobile_number":contact_dict['mobile_number'],"office_number":contact_dict['office_number'],'agent_license':agent_details['agent_licenses'], 'agent_bio':agent_details['agent_bio'],'serviceAreas':agent_details['serviceAreas'],'office_name':agent_details['agent_team_name'],'office_address':agent_details['agent_team_info'],'logo_url':agent_details['logo_url'],'facebook':social_media_dict['facebook'], 'instagram':social_media_dict['instagram'],'twitter':social_media_dict['twitter'],'linkedin':social_media_dict['linkedin'],'Market_Cente':other_info['Market_Cente'],'agent_language':other_info['Languages'],'Specialties_Designations':other_info['Specialties_Designations'],'social_media_links_str':social_media_links_str,'agent_website':contact_dict['agent_website']}
    data_df = pd.DataFrame(data_dict,index=[0],columns=['agent_url','agent_name', 'agent_role','location','agent_dp', 'tag','agent_email', 'mobile_number','office_number', 'agent_license', 'agent_bio', 'serviceAreas','office_name', 'office_address','logo_url','facebook', 'instagram', 'twitter', 'linkedin','Market_Cente', 'agent_language', 'Specialties_Designations','social_media_links_str','agent_website'])
    with open("kw_data-emailSelenium_43k_44k.csv",'a',newline='',encoding='utf-8') as f:
        data_df.to_csv(f, mode='a',header=f.tell()==0)
    print(url_count,url)
    url_count+=1
    driver_count+=1
    print("***********************************")
driver.quit()


https://kw.com/agent/UPA-6742537734547943424-8
Page load happened
43000 https://kw.com/agent/UPA-6742537734547943424-8
***********************************
https://kw.com/agent/UPA-6587385374922719233-0
Page load happened
43001 https://kw.com/agent/UPA-6587385374922719233-0
***********************************
https://kw.com/agent/UPA-6773024002891939840-2
Page load happened
43002 https://kw.com/agent/UPA-6773024002891939840-2
***********************************
https://kw.com/agent/UPA-6587385074641072134-5
Page load happened
43003 https://kw.com/agent/UPA-6587385074641072134-5
***********************************
https://kw.com/agent/UPA-6740092292165558272-3
Page load happened
43004 https://kw.com/agent/UPA-6740092292165558272-3
***********************************
https://kw.com/agent/UPA-6587385020532617222-9
Page load happened
43005 https://kw.com/agent/UPA-6587385020532617222-9
***********************************
https://kw.com/agent/UPA-6803100614250438656-3
Page load happened
4300

Page load happened
43053 https://kw.com/agent/UPA-6846812513520463872-6
***********************************
https://kw.com/agent/UPA-6785284838734778368-2
Page load happened
43054 https://kw.com/agent/UPA-6785284838734778368-2
***********************************
https://kw.com/agent/UPA-6597235428434726913-8
Page load happened
43055 https://kw.com/agent/UPA-6597235428434726913-8
***********************************
https://kw.com/agent/UPA-6809231208713773056-3
Page load happened
43056 https://kw.com/agent/UPA-6809231208713773056-3
***********************************
https://kw.com/agent/UPA-6587385340426125314-0
Page load happened
43057 https://kw.com/agent/UPA-6587385340426125314-0
***********************************
https://kw.com/agent/UPA-6587385212736126978-1
Page load happened
43058 https://kw.com/agent/UPA-6587385212736126978-1
***********************************
https://kw.com/agent/UPA-6841816029184663552-2
Page load happened
43059 https://kw.com/agent/UPA-6841816029184663552-

Page load happened
43106 https://kw.com/agent/UPA-6587385176533299204-0
***********************************
https://kw.com/agent/UPA-6719994821703622656-3
Page load happened
43107 https://kw.com/agent/UPA-6719994821703622656-3
***********************************
https://kw.com/agent/UPA-6762846943216590848-0
Page load happened
43108 https://kw.com/agent/UPA-6762846943216590848-0
***********************************
https://kw.com/agent/UPA-6772269002357657600-6
Page load happened
43109 https://kw.com/agent/UPA-6772269002357657600-6
***********************************
https://kw.com/agent/UPA-6587384978770268166-9
Page load happened
43110 https://kw.com/agent/UPA-6587384978770268166-9
***********************************
https://kw.com/agent/UPA-6587385010289385475-7
Page load happened
43111 https://kw.com/agent/UPA-6587385010289385475-7
***********************************
https://kw.com/agent/UPA-6587385180104146948-9
Page load happened
43112 https://kw.com/agent/UPA-6587385180104146948-

Page load happened
43159 https://kw.com/agent/UPA-6587385216005316610-4
***********************************
https://kw.com/agent/UPA-6587385180154478597-8
Page load happened
43160 https://kw.com/agent/UPA-6587385180154478597-8
***********************************
https://kw.com/agent/UPA-6693630792932495360-2
Page load happened
43161 https://kw.com/agent/UPA-6693630792932495360-2
***********************************
https://kw.com/agent/UPA-6698613335372869632-3
Page load happened
43162 https://kw.com/agent/UPA-6698613335372869632-3
***********************************
https://kw.com/agent/UPA-6587385445868511235-6
Page load happened
43163 https://kw.com/agent/UPA-6587385445868511235-6
***********************************
https://kw.com/agent/UPA-6587385209222860805-4
Page load happened
43164 https://kw.com/agent/UPA-6587385209222860805-4
***********************************
https://kw.com/agent/UPA-6808038846749245440-9
Page load happened
43165 https://kw.com/agent/UPA-6808038846749245440-

Page load happened
43212 https://kw.com/agent/UPA-6587385214982590468-4
***********************************
https://kw.com/agent/UPA-6592944824525553667-3
Page load happened
43213 https://kw.com/agent/UPA-6592944824525553667-3
***********************************
https://kw.com/agent/UPA-6801591401030483968-9
Page load happened
43214 https://kw.com/agent/UPA-6801591401030483968-9
***********************************
https://kw.com/agent/UPA-6773294615133663232-7
Page load happened
43215 https://kw.com/agent/UPA-6773294615133663232-7
***********************************
https://kw.com/agent/UPA-6587385431867269125-2
Page load happened
43216 https://kw.com/agent/UPA-6587385431867269125-2
***********************************
https://kw.com/agent/UPA-6733084608127692800-2
Page load happened
43217 https://kw.com/agent/UPA-6733084608127692800-2
***********************************
https://kw.com/agent/UPA-6722259425005469696-8
Page load happened
43218 https://kw.com/agent/UPA-6722259425005469696-

...

Page load happened
43742 https://kw.com/agent/UPA-6587385039448788992-2
***********************************
https://kw.com/agent/UPA-6587385354370191369-8
Page load happened
43743 https://kw.com/agent/UPA-6587385354370191369-8
***********************************
https://kw.com/agent/UPA-6592952010554937344-4
Page load happened
43744 https://kw.com/agent/UPA-6592952010554937344-4
***********************************
https://kw.com/agent/UPA-6846510525299175424-2
Page load happened
43745 https://kw.com/agent/UPA-6846510525299175424-2
***********************************
https://kw.com/agent/UPA-6592964453671477250-0
Page load happened
43746 https://kw.com/agent/UPA-6592964453671477250-0
***********************************
https://kw.com/agent/UPA-6772953153305534464-0
Page load happened
43747 https://kw.com/agent/UPA-6772953153305534464-0
***********************************
https://kw.com/agent/UPA-6587385343968002051-9
Page load happened
43748 https://kw.com/agent/UPA-6587385343968002051-

Page load happened
43795 https://kw.com/agent/UPA-6841408272378163200-1
***********************************
https://kw.com/agent/UPA-6587385283252056067-3
Page load happened
43796 https://kw.com/agent/UPA-6587385283252056067-3
***********************************
https://kw.com/agent/UPA-6661300081763938304-4
Page load happened
43797 https://kw.com/agent/UPA-6661300081763938304-4
***********************************
https://kw.com/agent/UPA-6587385449244872707-0
Page load happened
43798 https://kw.com/agent/UPA-6587385449244872707-0
***********************************
https://kw.com/agent/UPA-6587385395517677573-7
Page load happened
43799 https://kw.com/agent/UPA-6587385395517677573-7
***********************************
https://kw.com/agent/UPA-6587385433858826246-1
need to close driver
new driver initialize****************
Page load happened
43800 https://kw.com/agent/UPA-6587385433858826246-1
***********************************
https://kw.com/agent/UPA-6826626553818955776-2
Page load h

Page load happened
43848 https://kw.com/agent/UPA-6587385245593260036-9
***********************************
https://kw.com/agent/UPA-6587385002015002624-7
Page load happened
43849 https://kw.com/agent/UPA-6587385002015002624-7
***********************************
https://kw.com/agent/UPA-6747264367506604032-9
Page load happened
43850 https://kw.com/agent/UPA-6747264367506604032-9
***********************************
https://kw.com/agent/UPA-6587385277360787457-7
Page load happened
43851 https://kw.com/agent/UPA-6587385277360787457-7
***********************************
https://kw.com/agent/UPA-6587385330231164934-7
Page load happened
43852 https://kw.com/agent/UPA-6587385330231164934-7
***********************************
https://kw.com/agent/UPA-6587385163942395909-2
Page load happened
43853 https://kw.com/agent/UPA-6587385163942395909-2
***********************************
https://kw.com/agent/UPA-6663731069714829313-7
Page load happened
43854 https://kw.com/agent/UPA-6663731069714829313-

Page load happened
43901 https://kw.com/agent/UPA-6672116951460163584-7
***********************************
https://kw.com/agent/UPA-6587385072122793984-7
Page load happened
43902 https://kw.com/agent/UPA-6587385072122793984-7
***********************************
https://kw.com/agent/UPA-6587384994094403586-2
Page load happened
43903 https://kw.com/agent/UPA-6587384994094403586-2
***********************************
https://kw.com/agent/UPA-6587385229252681737-2
Page load happened
43904 https://kw.com/agent/UPA-6587385229252681737-2
***********************************
https://kw.com/agent/UPA-6587385252167864325-2
Page load happened
43905 https://kw.com/agent/UPA-6587385252167864325-2
***********************************
https://kw.com/agent/UPA-6726517468972761088-2
Page load happened
43906 https://kw.com/agent/UPA-6726517468972761088-2
***********************************
https://kw.com/agent/UPA-6587385338206351363-5
Page load happened
43907 https://kw.com/agent/UPA-6587385338206351363-

Page load happened
43954 https://kw.com/agent/UPA-6739518525878136832-4
***********************************
https://kw.com/agent/UPA-6587385305401180161-5
Page load happened
43955 https://kw.com/agent/UPA-6587385305401180161-5
***********************************
https://kw.com/agent/UPA-6587385193510522882-0
Page load happened
43956 https://kw.com/agent/UPA-6587385193510522882-0
***********************************
https://kw.com/agent/UPA-6715343830010015744-8
Page load happened
43957 https://kw.com/agent/UPA-6715343830010015744-8
***********************************
https://kw.com/agent/UPA-6587385366026498049-2
Page load happened
43958 https://kw.com/agent/UPA-6587385366026498049-2
***********************************
https://kw.com/agent/UPA-6793257820974514176-6
Page load happened
43959 https://kw.com/agent/UPA-6793257820974514176-6
***********************************
https://kw.com/agent/UPA-6620948969686503424-3
Page load happened
43960 https://kw.com/agent/UPA-6620948969686503424-